# Integration

Research in astrophysics frequently involves evaluating integrals numerically. The basic idea behind numerical "quadrature" is that a definite integral of a function can be approximated by a sum over evaluations of that function multiplied by quadrature "weights":

\begin{equation}
    \int_a^b f(x)\text{d}x \approx \sum_n w_n f(x_n), \hspace{1em} x_n\in[a,b]
\end{equation}

Depending on the function $f$, different choices of weights $w_n$ and grid-points $x_n$ can lead to improvements in accuracy (to floating-point error, in some cases). One of the simplest quadratures, known as the trapezoidal rule, approximates a definite integral as the sum of the areas of trapezoids defined by the endpoints of line segments along the curve defined by $f$:

\begin{align}
    \int_a^b f(x)\text{d}x 
    &\approx \sum_N \text{[area of trapezoid defined by the four points }(x_n,0),(x_n,f(x_n)),(x_{n+1},0),(x_{n+1},f(x_{n+1}))],
\\
    &=\sum_{n=0}^{N-2}\frac{1}{2}(x_{n+1}-x_n)[f(x_n) + f(x_{n+1})]
\end{align}
on an interval with $N$ grid-points $x_0,...,x_{N-1}$. For a grid of evenly spaced points with grid-size $\Delta x$, this simplifies further to $\Delta x[f(x_0)/2 + f(x_1) + ... + f(x_{N-2}) + f(x_{N-1})/2]$. Here's an example of a function that applies the trapezdoidal rule while  iteratively halving the grid-spacing until the integral converges:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def trapz_int(f,a,b,tol=1e-10,maxits=100):
    '''
        uses the trapezoidal rule to compute the definite integral 
        int_a^b f(x)dx, iteratively increasing the number of grid
        points used for quadrature until the solution converges
        
        Inputs:
            f       function to integrate
            a       starting point of integration
            b       en-point of integration
            tol     tolerance for change in subsequent iterations
            maxits  maximum number of iterations
        Outputs:
            I       definite integral int_a^b f(x)dx
    '''
    # evaluate initial approximation to the integral:
    dx= (b-a)                  # coursest possible grid-spacing
    I = dx*(f(b)/2. + f(a)/2.) # roughest approximation to integral
    N = 2                      # smallest number of grid-pointsS
    
    # iteratively refine by halving the grid-spacing:
    itr= 0
    while itr<maxits:
        dx_new = dx/2. # decrease grid-spacing
        N += 2**itr    # increase number of grid-points
        x_eval = np.linspace(a,b,N)[1:-1][::2] # this indexing takes only the grid-points we haven't evaluated yet
        I_new = I*dx_new/dx \
              + np.sum(f(x_eval))*dx_new
        print(f'iteration {itr} ({N} points): {100*(I_new-I)/I:.4e}% change')
        if abs(I_new-I)<tol:
            break        
        itr+=1
        dx = dx_new
        I = I_new

    if itr==maxits:
        print(f'integral did not converge after {itr} refinements')
    else:
        print('integral converged!')
    return I

Let's use this function to calculate the integral $\int_0^5 dx (x^2 + 3)$, which has the exact answer $5^3 / 3 + 3\times5\simeq 56.667$:

In [ ]:
%%time
ans = 5.**3 / 3 + 3 * 5.
tst = trapz_int(f=lambda x: x**2 + 3,a=0,b=5,tol=1e-10)
print(f'error of {(ans - tst)/ans*100:.4e}% ')

This converged reasonably well, but it required evaluating our function at a lot of points. Scipy includes a wrapper for a Fortran routine that uses a much more accurate, and also efficient quadrature (see https://en.wikipedia.org/wiki/Gaussian_quadrature):

In [ ]:
from scipy.integrate import quad
quad?

In [ ]:
%%time 
I, err = quad(func=lambda x: x**2 + 3, a=0, b=5, epsabs=1.e-15)
print (I, err)

### Integration of a pre-sampled function

In astrophysics we frequently need to calculate the area under the curve of a function pre-sampled at fixed points. The approach to integrating over a pre-sampled function isn't any different in principle, but we no longer have the option of iteratively refining our answer until the solution converges. 

### Exercise
Write a function, called trapz_fixed_int(f,x), that takes as input an array of samples $f$ of a function $f$ at an array of grid-points $x$, and uses the trapezoidal rule to compute the definite integral $\int_a^bf\text{d}x$, where $a$ and $b$ are the end-points $x[0]$ and $x[-1]$.

In [ ]:
xx = np.linspace(0,5,1000)

In [ ]:
%%time 
tst= trapz_fixed_int(xx**2 + 3,xx)
print(f'error of {(ans - tst)/ans*100:.4e}% ')

Scipy also includes some ready-to-use functions for integrating over pre-sampled points:

In [ ]:
from scipy.integrate import trapezoid,simpson

In [ ]:
%%time 
# this should give the same answer as trapz_fixed_int
tst= trapezoid(y=xx**2 + 3,x=xx)
print(f'error of {(ans - tst)/ans*100:.4e}% ')

In [ ]:
%%time 
tst= simpson(y=xx**2 + 3,x=xx)
print(f'error of {(ans - tst)/ans*100:.4e}% ')

# Interpolation

We often have data sampled at a relatively small number of points, and so we need to interpolate between them. There are a number of different ways to interpolate functions (many of which are very closely related to the numerical integration described above). Scipy has a few built-in options that do reasonably well:

In [ ]:
from scipy.interpolate import interp1d,CubicSpline,splrep,splev

Let's try interpolating the function $f(x) = e^{-x}  \sin(2  \pi  x)$ using only a small number of points:

In [ ]:
# define and sample the function at a small number of points
def myf(x):
    return (np.exp(-x)) * np.sin(2 * np.pi * x)
x_coarse = np.linspace(0,2,8)
data = myf(x_coarse)

# construct interpolation functions:
linterp = interp1d(x=x_coarse,y=data,kind='linear')
splterp = splrep(x=x_coarse,y=data)
csplterp= CubicSpline(x=x_coarse,y=data)

# evaluate and plot the interpolation functions
x_fine = np.linspace(0,2,1000) 
truth = myf(x_fine)

f,ax = plt.subplots(2,1,figsize=(10,8),sharex=True)
ax[0].plot(x_coarse,data,'ko')
ax[0].plot(x_fine,linterp(x_fine),label='linear')
ax[0].plot(x_fine,csplterp(x_fine),label='Spline #1')
ax[0].plot(x_fine,splev(x_fine,splterp),label='Spline #2')
ax[0].plot(x_fine,truth,'k',label='Truth')
ax[0].legend(fontsize=12)
ax[0].set_ylabel('$f(x)$')

# now plot residuals from the function evaluation:
for terp in [linterp(x_fine),csplterp(x_fine),splev(x_fine,splterp)]:
    ax[1].plot(x_fine,terp - truth)
ax[1].set_ylabel('Residual')
ax[1].set_xlabel('$x$')